In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import webdl_utils

In [6]:
import torch

enc = torch.load("../ChestXrays/ALI/model/AE_0.00001_1000_512/AE_0.00001_1000_512_GenZ_It_1600000.pth", map_location=torch.device('cpu'))
dec = torch.load("../ChestXrays/ALI/model/AE_0.00001_1000_512/AE_0.00001_1000_512_GenX_It_1600000.pth", map_location=torch.device('cpu'))


In [10]:
import sys
sys.path.append('../ChestXrays/ALI')
from ALImodel import *
from ALIloader import *

In [17]:
LS = 512
ColorsNumber = 1 #Number of color (always 1 for x-ray)
inputsize = 64
ExpDir = "/tmp/"
cp = -1 #Checkpoint to load (-2 for latest one, -1 for last epoch)

DisX,DisZ,DisXZ,GenZ,GenX,CP,DiscriminatorLoss,tAUCs = GenModel(
    inputsize,LS,cp,ExpDir,"test",ColorsNumber=ColorsNumber)

In [20]:
GenZ.load_state_dict(enc);
GenX.load_state_dict(dec);

In [27]:
dummy_input = torch.autograd.Variable(torch.FloatTensor(torch.zeros([1, 1, 64, 64])))
GenZ.eval()
torch.onnx.export(GenZ, dummy_input, 'enc_chest.onnx')
dummy_input = torch.autograd.Variable(torch.FloatTensor(torch.zeros([1, 512, 1, 1])))
GenX.eval()
torch.onnx.export(GenX, dummy_input, 'dec_chest.onnx')

In [30]:
webdl_utils.convert_onnx2tf_pb(onnx_file="enc_chest.onnx", pb_file="enc_chest.pb")
webdl_utils.convert_onnx2tf_pb(onnx_file="dec_chest.onnx", pb_file="dec_chest.pb")

onnx 1.5.0
onnx-tf 1.3.0
tensorflow 1.14.0


In [34]:
GenZ

Encoder(
  (main): Sequential(
    (Conv_0): Conv2d(1, 64, kernel_size=(2, 2), stride=(1, 1), bias=False)
    (LRelu_0): LeakyReLU(negative_slope=0.1, inplace=True)
    (BN_0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Conv_1): Conv2d(64, 128, kernel_size=(7, 7), stride=(2, 2), bias=False)
    (LRelu_1): LeakyReLU(negative_slope=0.1, inplace=True)
    (BN_1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Conv_2): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (LRelu_2): LeakyReLU(negative_slope=0.1, inplace=True)
    (BN_2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Conv_3): Conv2d(256, 512, kernel_size=(7, 7), stride=(2, 2), bias=False)
    (LRelu_3): LeakyReLU(negative_slope=0.1, inplace=True)
    (BN_3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Conv_4): Conv2d(512, 512, kernel_size=(4, 4), str

In [48]:
!rm -rf enc-chest-savedmodel
!rm -rf dec-chest-savedmodel

In [49]:
b = webdl_utils.convert_tf_pb2savedmodel(pb_file="enc_chest.pb", savedmodel_folder="enc-chest-savedmodel")


tensorflow 1.14.0
Possible inputs: [('input.1', 'Placeholder')]
Selecting first element in list: input.1:0
Possible outputs: [('transpose_17', 'Transpose')]
Selecting first element in list: transpose_17:0


In [51]:
b = webdl_utils.convert_tf_pb2savedmodel(pb_file="dec_chest.pb", savedmodel_folder="dec-chest-savedmodel")

tensorflow 1.14.0
Possible inputs: [('0', 'Placeholder')]
Selecting first element in list: 0:0
Possible outputs: [('Tanh', 'Tanh')]
Selecting first element in list: Tanh:0


In [53]:
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    enc-chest-savedmodel \
    enc-chest-savedmodel-tfjs

2019-11-16 17:35:22.376706: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-11-16 17:35:23.077526: I tensorflow/core/grappler/devices.cc:60] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA support)
2019-11-16 17:35:23.077630: I tensorflow/core/grappler/clusters/single_machine.cc:359] Starting new session
2019-11-16 17:35:23.206378: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:716] Optimization results for grappler item: graph_to_optimize
2019-11-16 17:35:23.206411: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:718]   function_optimizer: function_optimizer did nothing. time = 0.002ms.
2019-11-16 17:35:23.206419: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:718]   function_optimizer: function_optimizer did nothing. time = 0.001ms.
2019-11-16 17:35:26.072328: I tensorflow/core/grappler/op

In [54]:
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    dec-chest-savedmodel \
    dec-chest-savedmodel-tfjs

2019-11-16 17:35:34.232660: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-11-16 17:35:34.520448: I tensorflow/core/grappler/devices.cc:60] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA support)
2019-11-16 17:35:34.520548: I tensorflow/core/grappler/clusters/single_machine.cc:359] Starting new session
2019-11-16 17:35:34.547878: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:716] Optimization results for grappler item: graph_to_optimize
2019-11-16 17:35:34.547991: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:718]   function_optimizer: function_optimizer did nothing. time = 0.003ms.
2019-11-16 17:35:34.548004: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:718]   function_optimizer: function_optimizer did nothing. time = 0.001ms.
2019-11-16 17:35:35.414382: I tensorflow/core/grappler/op